**Directory Context**

Here we are, back in Cursor, going to the fifth directory where there are a few things for me to show you.

### **Template Agent: `agent.py`**

**Purpose**

* `agent.py` is our template.
* This file is given to another agent, asking it to use this as its model/example.
* The idea is to **take this as a template** and make other agents like this.
* It's simply a **prototype** to be cloned and varied — **by an agent, not by us**.

**Imports**

* The file does some necessary imports.

**System Message**

* Sets a system message describing the agent’s personality and mission:

  > “You are a creative entrepreneur. Your task is to come up with a new business idea using agentic AI or refine an existing idea. Your personal interests are in these sectors…”
* Drawn to disruption, less interested in pure automation, optimistic, adventurous, risk-taking, imaginative, impatient, sometimes impulsive.
* Should respond in an engaging and clear way.
* **Comment at the top:**
  “Change this system message to reflect the unique characteristics of this agent.”

**Behavior Constants**

* A constant:
  `chances that I bounce an idea off another is 0.5`
* **Comment:**
  “You can also change the code to make the behavior different, but be careful to keep the method signatures the same.”

**Initialization (`__init__`)**

* Sets `gpt40mini` as the LLM with a temperature of 0.7 (for more randomness).
* Creates a delegate assistant using this model client and the defined system message.

**Main Message Handler (`handleMyMessage` / `handleMessage`)**

* Decorated as a message handler.
* Takes a `message` (which uses the `messages` object, now separated into its own package for minimal code/reduced mistakes).
* Signature: takes a `messages.message`, returns a `messages.message`.
* Prints when it receives a message, including its type (like the agent’s name).

**Core Logic:**

1. Makes a text message from the input.
2. Sends this message to the underlying LLM (with the system prompt in context).
3. Waits for and receives the LLM response, treating it as a new business idea.
4. Picks a random number (`random.random()` between 0 and 1).

   * If below 0.5, will *bounce* the idea to another agent for refinement (using a utility function `find_recipient`).
   * Sends a message: “Here is my business idea. This might not be your specialty, but please refine it and make this business idea better.”
   * Calls `self.sendMessage()` (leveraging the runtime) to send to the randomly picked recipient.
   * Returns either its original idea, or a refined version, depending on the random outcome.

**Summary**

* **This agent is a template**:
  It can be cloned by other agents to create variants, each possibly with different personalities or behaviors, but with the same basic logic and message-handling capability.
* You can see how agents receive and send messages, sometimes collaborating (with some probability).
* It’s designed to allow dynamic, creative “agent creation” and interaction, with plenty of room for experimentation in multi-agent systems.


In [1]:
cat agent.py

from autogen_core import MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
import messages
import random


class Agent(RoutedAgent):

    # Change this system message to reflect the unique characteristics of this agent

    system_message = """
    You are a creative entrepreneur. Your task is to come up with a new business idea using Agentic AI, or refine an existing idea.
    Your personal interests are in these sectors: Healthcare, Education.
    You are drawn to ideas that involve disruption.
    You are less interested in ideas that are purely automation.
    You are optimistic, adventurous and have risk appetite. You are imaginative - sometimes too much so.
    Your weaknesses: you're not patient, and can be impulsive.
    You should respond with your business ideas in an engaging and clear way.
    """

    CHANCES_THA



### `messages.py` 

**Purpose**

* `messages.py` defines the core messaging logic and structures for agent communication.

**Key Points:**

* **Data Class:** Contains a familiar data class for the message object.
* **findRecipient:**

  * Function to select another agent to communicate with.
  * Very “hacky” logic:

    * As agents (clones) are created (agent1, agent2, agent3…), the function scans the directory for existing agents.
    * It randomly selects one of the available agent files.
    * There’s a chance the agent might talk to itself (by being selected as its own recipient).
    * This could be improved to avoid self-messaging, but is left as-is for now.


In [2]:
cat messages.py

from dataclasses import dataclass
from autogen_core import AgentId
import glob
import os


import random

@dataclass
class Message:
    content: str


def find_recipient() -> AgentId:
    try:
        agent_files = glob.glob("agent*.py")
        agent_names = [os.path.splitext(file)[0] for file in agent_files]
        agent_names.remove("agent")
        agent_name = random.choice(agent_names)
        print(f"Selecting agent for refinement: {agent_name}")
        return AgentId(agent_name, "default")
    except Exception as e:
        print(f"Exception finding recipient: {e}")
        return AgentId("agent1", "default")


### `creator.py`

* `creator.py` is the AGENT that *creates* and *spawns* new agents dynamically.

**Key Points:**

* **System Message:**

  * The agent receives instructions to generate new agent code from a template,
    using AutogenCore and AutogenAgentChat.
  * Flexibility to change the agent’s personality or system prompt.
  * The only requirements: the new class must be named `agent`, must inherit from `rooted agent`, and must have an `__init__` method.
  * Output must be only Python code.

* **How Creation Works:**

  * The creator agent reads `agent.py` as the template.
  * It asks an LLM to mutate the code, giving each agent a unique system message, and possibly other variations.
  * Each new agent is saved as `agent1.py`, `agent2.py`, etc.
  * Uses `importlib` to dynamically import the new agent module it just wrote.
  * Registers the newly imported agent with the runtime, using a lambda/factory for instantiation.
  * Declares the new agent “live”:

    * Not just “deployed” in the software sense, but *running* and ready to receive messages.
  * Immediately sends the new agent a message, triggering it to generate and possibly refine a business idea.

* **Agent Interaction:**

  * Agents can message each other, or sometimes themselves, for idea refinement.
  * Demonstrates true agent-to-agent messaging and autonomy.
  * Built on asynchronous Python for efficiency and concurrency.

**What Makes This Architecture Special?**

* **Dynamically extensible:** Agents *generate code*, spawn new agents, and register them at runtime.
* **Inter-process communication:**

  * No need to worry about networking or plumbing—the platform takes care of routing messages, even across runtimes.
* **Creativity and risk:**

  * The agents are free to mutate their own logic and personality, within some guardrails.
* **Async Python:**

  * The system is designed to scale and handle concurrent agent activity.

**What Next?**

* I can:

  * **Explain the exact logic in your files** (line-by-line or by function/class).
  * **Suggest improvements** (e.g., to avoid self-messaging).
  * **Visualize** the flow with diagrams.
  * **Help you adapt or extend** this for a specific project.

**What would you like to do next?**
For example:

* “Show me a code walkthrough for messages.py”
* “Suggest improvements for the findRecipient logic”
* “Explain the dynamic import in creator.py”
* “Show how to make it avoid self-messaging”
* Or, “Run an example interaction with agent creation and messaging (pseudo-code)”

Let me know!

In [3]:
cat creator.py

from autogen_core import MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
import messages
from autogen_core import TRACE_LOGGER_NAME
import importlib
import logging
from autogen_core import AgentId

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(TRACE_LOGGER_NAME)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.DEBUG)


class Creator(RoutedAgent):

    # Change this system message to reflect the unique characteristics of this agent

    system_message = """
    You are an Agent that is able to create new AI Agents.
    You receive a template in the form of Python code that creates an Agent using Autogen Core and Autogen Agentchat.
    You should use this template to create a new Agent with a unique system message that is different from the template,
    and reflects their unique char



## **`World.py`: The Orchestrator Script**

We're nearly there. I've just got one more thing to show you, which is `world.py`, which is quite short, which is the overall thing.

**Role of world.py**

`creator.py`—you have to send it, it's an agent, and you have to send Creator a message, like agent1, agent2, agent3, and it will create them. So this is where it all comes together.

`world.py` is **not an agent**. This is just a Python script. Everything else you've seen has been agents. This is just a Python script, but it uses async Python in quite an interesting way.

**The createAndMessage Coroutine**

So there is this async function, this coroutine, `createAndMessage`, which takes a worker and a creator ID, and an i, which is going to be the agent number. We have 1, 2, 3, 4, 5.
This "how many agents up here" is a pretty important one to look at. This is how many of these it's going to kick off at the same time.

**Cost and Model Risk**

And the other thing that's unsafe about this, of course, is cost. This is going to hit the real APIs, and for me, it's cost like a couple of cents when I've done this for 20 agents, which is pretty cheap all things considered.
And obviously, if you do it for like three agents, it costs nothing. But there's a risk there that we are allowing—in theory—the agent creator could change the model requirements. It could decide to use an expensive model instead of GPT-4 over any.
And so that's just something to watch for. It's very unlikely, but it's not like it's not impossible.

**Async Method: Creating and Messaging Agents**

Anyways, we've got 20 agents here, so we've got this async method, creator, and endMessage. It takes this worker and it sends a message to the agent number whatever dot dot pi, and then it writes the result to idea whatever dot marker. And that's all this thing here does.

**Main Async Function: Event Loop and Parallelism**

Okay, so now we go to our main async method and our main async care function, our covert team. So it creates a host, a gRPC worker agent runtime host, as we did in the notebook before. It starts the host, it creates a worker, and it starts the worker.

It then registers the creator. So it registers the creator from creator.py that was imported here. So that will launch a creator. And this is its ID, creator default.

**Parallel Execution with asyncio.gather**

Then this is the async thing. So I don't want to have a loop and call this creating worker after worker, sorry, creating agent after agent. Because then it will be serial. It will happen one after another.
If I do await and then I call this coroutine, then we'll be waiting there. It will create one file, it will send it a message, back will come the answer, maybe it will send it somewhere else, and then it will go on to the next. And we'll be sitting here waiting, and it won't be exciting.

But what you can do with coroutines, as you hopefully remember from when we briefly talked about this, is that you can get a whole stash of coroutines together. So I haven't got the word await here, which you need if this is actually going to run.
I'm just gathering a whole list of coroutines. And then I await them using `asyncIO.gather`. And `asyncIO.gather` runs them all in parallel.

**Difference from Multithreading**

And as hopefully you remember, it's not like multithreading. They're not going to be actually running on different threads, like with the CPU chopping between them.
But rather, they're running in the event loop. That's how asyncIO Python works. They run an event loop so that every time it's waiting on OpenAI, which means it's waiting on a network connection, another one can be running.
And that means they all get to run at the same time. As long as I stay within my rate limits with OpenAI, we'll be able to run plenty in parallel.

**Completion and Exception Handling**

And so it does all of that, and then at the end it stops, and any exceptions, it will print them. And that's it.

**How to Run async Python**

This is, by the way, how you run asyncPython from the command line or from a Python module. Your main file needs to do this, which brings, which initiates an `asyncIO` event loop and then runs your async method, your coroutine.

So that is how it hangs together.

**Project Summary**

And I do believe this is it. I don't think I've got anything else to show you. I think you've seen everything.
You've seen agent, which is the prototype, the thing that gets cloned. Creator, which does the cloning agent by agent. It's called one by one. And world.py is this.
It's not an agent. It's just Python code, which will orchestrate, which will launch the whole process running. Okay.



In [4]:
cat world.py

from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost
from agent import Agent
from creator import Creator
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime
from autogen_core import AgentId
import messages
import asyncio

HOW_MANY_AGENTS = 20

async def create_and_message(worker, creator_id, i: int):
    try:
        result = await worker.send_message(messages.Message(content=f"agent{i}.py"), creator_id)
        with open(f"idea{i}.md", "w") as f:
            f.write(result.content)
    except Exception as e:
        print(f"Failed to run worker {i} due to exception: {e}")

async def main():
    host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
    host.start() 
    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    result = await Creator.register(worker, "Creator", lambda: Creator("Creator"))
    creator_id = AgentId("Creator", "default")
    coroutines = [create_and_message(worker, creator_id, i) for i in range